In [103]:
from edgar import Company, TXTML, Edgar



#https://www.sec.gov/Archives/edgar/data/891103/000089110319000006/0000891103-19-000006-index.htm


company = Company("IAC/INTERACTIVECORP", "0000891103")
#doc = company.get_10K()
#doc_text = TXTML.get_HTML_from_document(doc)
#print(doc_text)
#text = TXTML.parse_full_10K(doc)

text = company._get_filings_url()
#https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000891103&type=&dateb=&owner=include&count=100



#text = company._group_document_type()
test21 = company.get_document_type_from_10K("EX-21.1")
#text = test21[0]._get_filings_url()
#print(test21)
#dir(TXTML)
#text = TXTML.get_document_type(test21[0])


text = TXTML.get_HTML_from_document(test21[0])
# iac-ex211_20181231.htm


#text = TXTML.parse_full_10K(test21[0])

#print(test21)
#print(text)
#dir(TXTML)
#dir(Company)
dir(Edgar)

#edgar = Edgar()
#possible_companies = edgar.find_company_name("Cisco System")
cik = edgar.get_cik_by_company_name('General Electic Inc.')

print(cik)

KeyError: 'General Electic Inc.'

In [108]:
cik = edgar.get_cik_by_company_name('General Electric Co')
print(cik)

KeyError: 'General Electric Co'

In [ ]:
import edgar
import inspect
import random
dir(edgar)
company2 = edgar.Company("IAC/INTERACTIVECORP", "0000891103")

In [ ]:
states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

class CompanyNode:
    def __init__(self, name, location, parent = None):
        self.name = name
        
        #tries to find CIK on SEC website
        try:
            self.cik = edgar.get_cik_by_company_name(self.name)
            self.SEC = True
        except:
            self.cik = Noneself.sic = None        
        
        self.location = location

        self.corp_type = eval_corp_type()
        
        self.parent = parent
        self.child = {}
        
        
        
        self.SEC = eval_SEC() #indicates whether the company can be found on SEC
        

    def eval_corp_type(self):
        corp_check = self.name.split()
        print(corp_check[-1])
        return corp_check[-1]
    
    def set_SIC(self, SIC):
        self.sic = SIC
    
    def eval_SEC(self):
        if self.location in states and cik != None:
            self.SEC = True
        else:
            self.SEC = False
    
    def set_child

# Process


Node Requirements:
- Name(manual or from list)
- CIK(get_cik_by_company_name)

Use Company and CIK to set company:
- EX 21.1 Document
- Use URL/Parse 21.1
- Get 10K (Optional)

Read Ex 21.1
- Get list of child companies
    - Get Name
    - Get Location

- SIC()
- Corp Type (Found through subsidiary document of parent)
- Location(Found through subsidiary document of parent)
- Parent (Found through subsidiary document of parent)
- Child (found through subsidiary document of self)

## Search Company Name in Edgar

## 